## Gram-Schmidt and Page Rank
### Submission preparation instructions 
_Completion of this header is mandatory, subject to a 2-point deduction to the assignment._ Only add plain text in the designated areas, i.e., replacing the relevant 'NA's. You must fill out all group member Names and Drexel email addresses in the below markdown list, under header __Module submission group__. It is required to fill out descriptive notes pertaining to any tutoring support received in the completion of this submission under the __Additional submission comments__ section at the bottom of the header. If no tutoring support was received, leave NA in place. You may as well list other optional comments pertaining to the submission at bottom. _Any distruption of this header's formatting will make your group liable to the 2-point deduction._

### Module submission group
- Group member 1
    - Name: Ruchita Paithankar
    - Email: 
- Group member 2
    - Name: 
    - Email: 
- Group member 3
    - Name: 
    - Email:
- Group member 4
    - Name: NA
    - Email: NA

### Additional submission comments
- Tutoring support received: NA
- Other (other): NA

## 1. Gram-Schmidt (10 points)

In [7]:
import numpy as np
import numpy.linalg as la

tol = 1e-14 # That's 1×10⁻¹⁴ = 0.00000000000001

# Our first function will perform the Gram-Schmidt procedure for 4 basis vectors.
# Go through the vectors one at a time and set them to be orthogonal
# to all the vectors that came before it. 
# Normalising or making it unit vector.

def orthoBasis4(A) :
    B = np.array(A,dtype=np.float_) 
    # First column is easy.. just normalise it. I.e. divide by its norm.
    B[:, 0] = B[:, 0] / la.norm(B[:, 0])
    # For second column, we need to subtract the projection of the first vector on the second
    B[:, 1] = B[:, 1] - B[:, 1] @ B[:, 0] * B[:, 0]
    # Note that if it is non zero, then B[:, 1] is linearly independant of B[:, 0]
    # If this is the case, we can normalise it. Otherwise we'll set that vector to zero.
    if la.norm(B[:, 1]) > tol :
        B[:, 1] = B[:, 1] / la.norm(B[:, 1])
    else :
        B[:, 1] = np.zeros_like(B[:, 1]) #zeros_like is similar to np.zeros but uses the dimension of input array
    # Now we need to repeat the process for remaining columns
    # Insert your code here. you can do it in 2 lines or one line
    B[:,2] = B[:,2] - B[:, 2] @ B[:, 0] * B[:, 0]
    B[:,2] = B[:,2] - B[:, 2] @ B[:, 1] * B[:, 1]
        
    
    # Again we'll need to normalise our new vector.
    # Repeat what we had done earlier
    
    if la.norm(B[:,2]) > tol :
        B[:, 2] = B[:, 2] / la.norm(B[:, 2])
    else :
        B[:, 2] = np.zeros_like(B[:, 2])
    
    
    # Finally, column three:
    # Insert your code here. either one line or three lines
    B[:,3] = B[:,3] - B[:, 3] @ B[:, 0] * B[:, 0]
    B[:,3] = B[:,3] - B[:, 3] @ B[:, 1] * B[:, 1]
    B[:,3] = B[:,3] - B[:, 3] @ B[:, 2] * B[:, 2]
    
    
    # Now normalise if possible
    if la.norm(B[:,3]) > tol :
        B[:, 3] = B[:, 3] / la.norm(B[:, 3])  
    else :
        B[:, 3] = np.zeros_like(B[:, 3])
  
    
    
    # Finally, we return the result:
    return B

# Exploit the repetition in the above code to generalize the function for any given matrix
def orthoBasis(A) :
    B = np.array(A, dtype=np.float_) 
    # Loop over all vectors, starting with zero, label them with i
    for i in range(B.shape[1]) :
        # Inside that loop, loop over all previous vectors, j, to subtract.
        for j in range(i) :
            # Complete the code
             B[:, i] = B[:,i] - B[:,i] @ B[:,j] * B[:,j]
        # Next insert code to do the normalisation test for B[:, i]
        if la.norm(B[:, i]) > tol :
            B[:, i] = B[:, i] / la.norm(B[:, i])  
        else :
            B[:, i] = np.zeros_like(B[:, i]) 
        
            
    # Finally, we return the result:
    return B



## 2. PageRank (5 points)
In this notebook, you'll build on your knowledge of eigenvectors and eigenvalues by exploring the PageRank algorithm.


### Introduction

PageRank (developed by Larry Page and Sergey Brin) revolutionized web search by generating a
ranked list of web pages based on the underlying connectivity of the web. The PageRank algorithm is
based on an ideal random web surfer who, when reaching a page, goes to the next page by clicking on a
link. The surfer has equal probability of clicking any link on the page and, when reaching a page with no
links, has equal probability of moving to any other page by typing in its URL. In addition, the surfer may
occasionally choose to type in a random URL instead of following the links on a page. The PageRank is
the ranked order of the pages from the most to the least probable page the surfer will be viewing.


In [8]:
import numpy as np
import numpy.linalg as la
np.set_printoptions(suppress=True)

### PageRank as a linear algebra problem
Let's imagine a micro-internet, with just 6 websites 
Each website links to some of the others, and this forms a network as shown,

![A Micro-Internet](internet_1.png "A Micro-Internet")


Imagine 100s of a person P on our micro-internet, each viewing a single website at a time.
Each minute P follow a link on their website to another site on the micro-internet.
After a while, the websites that are most linked to will have more Ps visiting them, and in the long run, each minute for every P that leaves a website, another will enter keeping the total numbers of Ps on each website constant.
The PageRank is simply the ranking of websites by how many Ps they have on them at the end of this process.

We represent the number of Ps on each website with the vector,
$$\mathbf{r} = \begin{bmatrix} r_A \\ r_B \\ r_C \\ r_D \\ r_E \\ r_F \end{bmatrix}$$
And say that the number of Ps on each website in minute $i+1$ is related to those at minute $i$ by the matrix transformation

$$ \mathbf{r}^{(i+1)} = L \,\mathbf{r}^{(i)}$$
with the matrix $L$ taking the form,
$$ L = \begin{bmatrix}
L_{A→A} & L_{B→A} & L_{C→A} & L_{D→A} & L_{E→A} & L_{F→A} \\
L_{A→B} & L_{B→B} & L_{C→B} & L_{D→B} & L_{E→B} & L_{F→B} \\
L_{A→C} & L_{B→C} & L_{C→C} & L_{D→C} & L_{E→C} & L_{F→C} \\
L_{A→D} & L_{B→D} & L_{C→D} & L_{D→D} & L_{E→D} & L_{F→D} \\
L_{A→E} & L_{B→E} & L_{C→E} & L_{D→E} & L_{E→E} & L_{F→E} \\
L_{A→F} & L_{B→F} & L_{C→F} & L_{D→F} & L_{E→F} & L_{F→F} \\
\end{bmatrix}
$$
where the columns represent the probability of leaving a website for any other website, and sum to one.
The rows determine how likely you are to enter a website from any other, though these need not add to one.
The long time behaviour of this system is when $ \mathbf{r}^{(i+1)} = \mathbf{r}^{(i)}$, so we'll drop the superscripts here, and that allows us to write,
$$ L \,\mathbf{r} = \mathbf{r}$$

which is an eigenvalue equation for the matrix $L$, with eigenvalue 1 (this is guaranteed by the probabalistic structure of the matrix $L$).

Complete the matrix $L$ below.

In [9]:
# Replace the ??? here with the probability of clicking a link to each website.
L = np.array([[0,   1/2, 1/3, 0, 0,   0   ],
              [1/3, 0,   0,   0, 1/2, 0   ],
              [1/3, 1/2, 0,   1, 0,   1/2 ],
              [1/3, 0,   1/3, 0, 1/2, 1/2 ],
              [0,   0,   0,   0, 0,   0   ],
              [0,   0,   1/3, 0, 0,   0   ]])

In principle, we could use a linear algebra library, as below, to calculate the eigenvalues and vectors.
And this would work for a small system. But this gets unmanagable for large systems.
And since we only care about the principal eigenvector (the one with the largest eigenvalue, which will be 1 in this case), we can use the *power iteration method* which will scale better, and is faster for large systems.

Since this is a small system, we could as well calculate the eigen vector.

In [10]:
eVals, eVecs = la.eig(L) # Gets the eigenvalues and vectors
order = np.absolute(eVals).argsort()[::-1] # Orders them by their eigenvalues
eVals = eVals[order]
eVecs = eVecs[:,order]

r = eVecs[:, 0] # Sets r to be the principal eigenvector
100 * np.real(r / np.sum(r)) # Make this eigenvector sum to one, then multiply by 100 Procrastinating Pats

array([16.        ,  5.33333333, 40.        , 25.33333333,  0.        ,
       13.33333333])

We can see from this list, the number of Ps that we expect to find on each website after long times.
Putting them in order of *popularity* (based on this metric), the PageRank of this micro-internet is:

**C**, **D**, **A**, **F**, **B**, **E**

Referring back to the micro-internet diagram, is this what you would have expected?
Convince yourself that based on which pages seem important given which others link to them, that this is a sensible ranking.

Let's now try to get the same result using the Power-Iteration method that we covered in the class.
This method will be much better at dealing with large systems.

First let's set up our initial vector, $\mathbf{r}^{(0)}$, so that we have our 100 Ps equally distributed on each of our 6 websites.

In [11]:
r = 100 * np.ones(6) / 6 # Sets up this vector (6 entries of 1/6 × 100 each)
r # Shows it's value

array([16.66666667, 16.66666667, 16.66666667, 16.66666667, 16.66666667,
       16.66666667])

Next, let's update the vector to the next minute, with the matrix $L$.
Run the following cell multiple times, until the answer stabilises.

In [12]:
r = L @ r # Apply matrix L to r
r # Show it's value
# Re-run this cell multiple times to converge to the correct answer.

array([13.88888889, 13.88888889, 38.88888889, 27.77777778,  0.        ,
        5.55555556])

We can automate applying this matrix multiple times as follows,

In [13]:
r = 100 * np.ones(6) / 6 # Sets up this vector (6 entries of 1/6 × 100 each)
for i in np.arange(100) : # Repeat 100 times
    r = L @ r
r

array([16.        ,  5.33333333, 40.        , 25.33333333,  0.        ,
       13.33333333])

Or even better, we can keep running until we get to the required tolerance.

In [14]:
r = 100 * np.ones(6) / 6 # Sets up this vector (6 entries of 1/6 × 100 each)
lastR = r
r = L @ r
i = 0
while la.norm(lastR - r) > 0.0001 :
    lastR = r
    r = L @ r
    i += 1
print(str(i) + " iterations to convergence.")
r

27 iterations to convergence.


array([15.99999654,  5.33332932, 40.00003127, 25.33332012,  0.        ,
       13.33332275])

See how the PageRank order is established fairly quickly, and the vector converges on the value we calculated earlier after a few tens of repeats.

### Damping Parameter
The system we just studied converged fairly quickly to the correct answer.
Let's consider an extension to our micro-internet where things start to go wrong.

Say a new website is added to the micro-internet: *G* Website.
This website is linked to by *FaceSpace* and only links to itself.
![An Expanded Micro-Internet](internet_2.png "An Expanded Micro-Internet")

Intuitively, only *F*, which is in the bottom half of the page rank, links to this website amongst the two others it links to,
so we might expect *G* site to have a correspondingly low PageRank score.

Build the new $L$ matrix for the expanded micro-internet, and use Power-Iteration.

In [15]:
 # We'll call this one L2, to distinguish it from the previous L.
L2 = np.array([[0,   1/2, 1/3, 0, 0,   0,   0 ],
               [1/3, 0,   0,   0, 1/2, 0,   0 ],
               [1/3, 1/2, 0,   1, 0,   1/3, 0 ],
               [1/3, 0,   1/3, 0, 1/2, 1/3, 0 ],
               [0,   0,   0,   0, 0,   0,   0 ],
               [0,   0,   1/3, 0, 0,   0,   0 ],
               [0,   0,   0,   0, 0,   1/3, 1 ]])

In [31]:
r = 100 * np.ones(7) / 7 # Sets up this vector (6 entries of 1/6 × 100 each)
lastR = r
r = L2 @ r
i = 0
while la.norm(lastR - r) > 0.0001 :
    lastR = r
    r = L2 @ r
    i += 1
print(str(i) + " iterations to convergence.")
r

229 iterations to convergence.


array([ 0.00031063,  0.0001085 ,  0.00072654,  0.00045093,  0.        ,
        0.00025378, 99.99814961])

That's no good! *G* seems to be taking all the traffic on the micro-internet, and somehow coming at the top of the PageRank.
This behaviour can be understood, because once a P get's to *G* Website, they can't leave, as all links head back to Geoff.

To combat this, we can add a small probability that the Ps don't follow any link on a webpage, but instead visit a website on the micro-internet at random.
We'll say the probability of them following a link is $d$ and the probability of choosing a random website is therefore $1-d$.
We can use a new matrix to work out where the Pat's visit each minute.
$$ M = d \, L + \frac{1-d}{n} \, J $$
where $J$ is an $n\times n$ matrix where every element is one.

If $d$ is one, we have the case we had previously, whereas if $d$ is zero, we will always visit a random webpage and therefore all webpages will be equally likely and equally ranked.
For this extension to work best, $1-d$ should be somewhat small - though we won't go into a discussion about exactly how small.

Let's retry this PageRank with this extension.

In [17]:
d = 0.85 # Feel free to play with this parameter after running the code once.
M = d * L2 + (1-d)/7 * np.ones([7, 7]) # np.ones() is the J matrix, with ones for each entry.

In [18]:
r = 100 * np.ones(7) / 7 # Sets up this vector (6 entries of 1/6 × 100 each)
lastR = r
r = M @ r
i = 0
while la.norm(lastR - r) > 0.01 :
    lastR = r
    r = M @ r
    i += 1
print(str(i) + " iterations to convergence.")
r

29 iterations to convergence.


array([11.65489945,  6.35616623, 24.03389289, 15.70372502,  2.14285714,
        8.95333321, 31.15512605])

This is certainly better, the PageRank gives sensible numbers for the Procrastinating Pats that end up on each webpage.
This method still predicts Geoff has a high ranking webpage however.
This could be seen as a consequence of using a small network. We could also get around the problem by not counting self-links when producing the L matrix (an if a website has no outgoing links, make it link to all websites equally).
We won't look further down this route, as this is in the realm of improvements to PageRank, rather than eigenproblems.

You are now in a good position, having gained an understanding of PageRank, to produce your own code to calculate the PageRank of a website with thousands of entries.

Good Luck!

## Exercise
Produce a function that can calculate the PageRank for an arbitrarily large probability matrix.


In [19]:
# Input link matrix L (which can be generated using the code given below) and damping factor d.
def pageRank(L, d) :
    n = L.shape[0]
    M = d * L + (1-d)/n * np.ones([n, n])
    r = 100 * np.ones(n) / n
    
    lastR = r
    r = M @ r
    
    while la.norm(lastR - r) > 0.01:
        lastR = r
        r = M @ r
    
    return r
    


In [20]:
#generatea random internet links
def generate_link(n) :
    c = np.full([n,n], np.arange(n))
    c = (abs(np.random.standard_cauchy([n,n])/2) > (np.abs(c - c.T) + 1))+0
    c = (c+1e-10) / np.sum((c+1e-10), axis=0)
    return c

In [21]:
# Use the following function to generate internets of different sizes.
generate_link(5)

array([[0.5, 0.2, 0. , 0.2, 0. ],
       [0. , 0.2, 0.5, 0.2, 0. ],
       [0.5, 0.2, 0.5, 0.2, 0. ],
       [0. , 0.2, 0. , 0.2, 1. ],
       [0. , 0.2, 0. , 0.2, 0. ]])

In [22]:
# Test your PageRank method against the built in "eig" method.
L = generate_link(5)

In [23]:
pageRank(L, 1)

array([99.97329202,  0.00549218,  0.0078618 ,  0.00549218,  0.0078618 ])

In [27]:
# using eigen vectors as seen before.
eVals, eVecs = la.eig(L) # Gets the eigenvalues and vectors
order = np.absolute(eVals).argsort()[::-1] # Orders them by their eigenvalues
eVals = eVals[order]
eVecs = eVecs[:,order]

r = eVecs[:, 0]
100 * np.real(r / np.sum(r))

array([99.99999982,  0.00000004,  0.00000005,  0.00000004,  0.00000005])

In [80]:
#your answers do not match always. Why?